<a target="_blank" href="https://colab.research.google.com/github/heechul/EmbeddedML/blob/main/l5-vww/train-person.ipynb">
        <img src="https://i.ibb.co/2P3SLwK/colab.png"  style="padding-bottom:5px;" />Run in Google Colab</a>

# Person-Detection using Transfer Learning
In this assignment we will use transfer learning to detect if there is a human in a given image. We will use a pre-trained model (MobileNet-V2), which was trained on ImageNet dataset and use a [Kaggle human detection dataset](https://www.kaggle.com/datasets/constantinwerner/human-detection-dataset).

**Acknowledgements**

This colab is heavily borrowed from the [Google's Transfer learning](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/images/transfer_learning.ipynb) Colab and [EdX TinyML course's](https://github.com/tinyMLx) Mask Detection Colab.


In [ ]:
!pip install -U tf_keras  &> 0 # Keras 2
!pip install kagglehub

import os
os.environ["TF_USE_LEGACY_KERAS"] = "1"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

: 

In [31]:
import matplotlib.pyplot as plt
import os
import tensorflow as tf

from tensorflow.keras.preprocessing import image_dataset_from_directory


# Suppress only PNG warnings
tf.get_logger().setLevel('ERROR')

RunningInCOLAB = 'google.colab' in str(get_ipython())

## Build the dataset

### Download the dataset

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("constantinwerner/human-detection-dataset")
print("Path to dataset files:", path)

In [ ]:
if RunningInCOLAB:
    !mv {path} /content/data
    !ls /content/data
    path = "/content/data"
else:
    !mv {path} data
    !ls data
    path = "./data"    
dataset_path = os.path.join(path, "human detection dataset")
print ("Path to dataset files:", dataset_path)

In [ ]:
topdir = os.getcwd()
if not os.path.exists('model'):
    os.mkdir('model')
if not os.path.exists('src'):
    os.mkdir('src')
print(topdir)

In [ ]:
import os
from tensorflow.keras.utils import image_dataset_from_directory

BATCH_SIZE = 32
IMG_SIZE = (96, 96)
dataset = image_dataset_from_directory(dataset_path,
                                        labels='inferred',
                                        label_mode='binary',
                                        shuffle=True,
                                        batch_size=BATCH_SIZE,
                                        image_size=IMG_SIZE)

dataset_size = len(list(dataset))  # Get the total number of elements in the dataset
train_size = int(0.8 * dataset_size) # Calculate the size of the training set (80%)
val_size = int(0.2 * dataset_size) # Calculate the size of the validation set (20%)

# Split the dataset into training and validation sets
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size).take(val_size)

Now that we have built the dataset lets view the first nine images and labels from the training set:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

class_names = dataset.class_names
print (class_names)
plt.figure(figsize=(10, 10))
for images, labels in dataset.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    idx = labels[i].numpy().astype("uint8")[0]
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[idx])
    plt.axis("off")

### Rescale pixel values

In a moment, you will download `tf.keras.applications.MobileNetV2` for use as your base model. This model expects pixel vaues in `[-1,1]`, but at this point, the pixel values in your images are in `[0-255]`. To rescale them, use the preprocessing method included with the model.

In [37]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

Note: Alternatively, you could rescale pixel values from `[0,255]` to `[-1, 1]` using a [Rescaling](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/Rescaling) layer.

Note: If you go on to use other `tf.keras.applications`, be sure to check the API doc to determine if they expect pixels in `[-1,1]` or `[0,1]`, or use the included `preprocess_input` function.

## Build the Model

### Create the base model
You will create the base model from the **MobileNet V2** model developed at Google. This is pre-trained on the ImageNet dataset, a large dataset consisting of 1.4M images and 1000 classes. ImageNet is a research training dataset with a wide variety of categories including objects such as `jackfruit` and `syringe`. This base of knowledge will help us classify if a person exists or not from our specific dataset.

First, you need to pick which layer of MobileNet V2 you will leverage as the high level features you wish to re-use. Since we want to adapt the classifications coming out of the model to a new task, we want to leverage the features coming out of the *last* layer BEFORE the classification layers. In many image models this is the output of the final convolution BEFORE the flatten layer. You may see this layer referred to as the "bottleneck layer" in some texts. Since many machine learning models are defined as the inputs occuring at the bottom and the outputs occuring at the top we would like to ignore the top few classification layers. Fortuntately, there is a shortcut to doing this in TensorFlow, **```include_top=False```**. By passing in this parameter we instantiate a MobileNet V2 model pre-loaded with weights trained on ImageNet that doesn't include the classification layers at the top, which is ideal for feature extraction.

In [38]:
# Create the base model from the pre-trained model MobileNet V2
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               alpha=0.35,
                                               weights='imagenet')

In [ ]:
image_batch, label_batch = next(iter(train_dataset))
feature_batch = base_model(image_batch)
print(feature_batch.shape)

### Freeze the convolutional layers

It is important to freeze the convolutional layers before you compile and train the model with transfer learning. Freezing (by setting layer.trainable = False) prevents the weights in a given layer from being updated during training. We want to perform this operation because we want to leverage the pre-trained values in the convolutional layers and only learn new classification layer values. We can do this by setting the entire model's `trainable` flag to False.

In [ ]:
print (base_model.trainable)
base_model.trainable = False
print (base_model.trainable)


In [ ]:
# Let's take a look at the base model architecture
base_model.summary()

### Add a classification head

To begin the process of generating classifications from the pretrained features, we use a `tf.keras.layers.GlobalAveragePooling2D` layer to convert the 3x3 spatial features into a single 1024-element feature vector per image.

In [ ]:
classification_input_layer = tf.keras.layers.Flatten()
# classification_input_layer = tf.keras.layers.GlobalAveragePooling2D()

feature_batch_average = classification_input_layer(feature_batch)
print(feature_batch_average.shape)

We then apply a `tf.keras.layers.Dense` layer to convert the feature vector into a single prediction per image. You don't need an activation function here because this prediction will be treated as a `logit`, or a raw prediction value.  Positive numbers predict class 1, negative numbers predict class 0.

In [ ]:
prediction_layer = tf.keras.layers.Dense(1, activation='sigmoid')
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

We can then build our final model by chaining together the data augmentation, rescaling, base_model and feature extractor layers using the [Keras Functional API](https://www.tensorflow.org/guide/keras/functional). Importantly we remind Tensorflow that we do not want to train the base_model!

In [44]:
inputs = tf.keras.Input(shape=(96, 96, 3))
x = preprocess_input(inputs)
x = base_model(x, training=False)
x = classification_input_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

### Compile the model

Compile the model before training it. Since there are two classes, use a binary cross-entropy loss with `from_logits=True` since the model provides a linear output. You can see below how we have included the Mobilenet V1 model after the input layer and before our classification layers!

In [ ]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [ ]:
model.summary()

## Your Turn: Train and Evaluate the model

Now that we have our model we can train it! You will see that since we are leveraging all of the pre-trained features we can improve our model from a random initialization (accuracy of ~50%) to a model with over 95% accuracy quite quickly. **How many epochs do you think we'll need?**

### Train the model
First print the initial accuracy

In [ ]:
loss, accuracy = model.evaluate(val_dataset)
print('Test accuracy :', accuracy)

**Now its your turn to pick the number of epochs of training! Remmeber we are aiming for at least 95% accuracy on the test set!**

*Hint: Despite the fact that it would take more than a day to train this model from scratch, it requires far fewer epochs to train it with transfer learning than you might suspect! You probably have more fingers and toes than the number of epochs you will need!*

In [ ]:
EPOCHS = 20  
history = model.fit(train_dataset,
                    epochs=EPOCHS,
                    validation_data=val_dataset,
                    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)])


Did you hit you accuracy goal? Did you overshoot and spend some extra time training? When might you have been able to quit? Lets take a look at the learning curves of the training and validation accuracy/loss to analyze our results.

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
# plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

### Evaluate your model

The last thing we need to do is check if the model is overfitting or if it actually learned the problem that fast! Does you model still perform well on the test set? Lets find out!

In [ ]:
loss, accuracy = model.evaluate(val_dataset)
print('Test accuracy :', accuracy)

And assuming your model passed the accuracy threshold you are now are all set to use this model to predict if the person exists or not. Lets print the results from a bunch of the images in our test dataset!


In [ ]:
#Retrieve a batch of images from the test set
image_batch, label_batch = val_dataset.as_numpy_iterator().next()
predictions = model.predict_on_batch(image_batch).flatten()

predictions = tf.where(predictions < 0.5, 0, 1)

print('Predictions:\n', predictions.numpy())
print('Labels:\n', label_batch.reshape(-1).astype("uint8"))

plt.figure(figsize=(10, 10))
for i in range(9):
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow(image_batch[i].astype("uint8"))
  plt.title(class_names[predictions[i]])
  plt.axis("off")

## Summary

* **Using transfer learning for Person Detection**: In this lab, we learned how we can use transfer learning to detect if a person exists or not. When working with a small dataset, it is a common practice to take advantage of features learned by a model trained on a larger dataset in the same domain.



# Model conversion

## Original float model

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT] # dynamic range quantization. size reduction but computing is done in fp32
tflite_model = converter.convert()

import pathlib
tflite_model_file = pathlib.Path('model/model-float32.tflite')
tflite_model_file.write_bytes(tflite_model)



## Quantized model

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]  

def representative_data_get():
    for input_value, _ in val_dataset.take(100):
        yield [input_value]

converter.representative_dataset = representative_data_get
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
tflite_model_quant = converter.convert()
tflite_model_quant_file = pathlib.Path('model/model-int8.tflite')
tflite_model_quant_file.write_bytes(tflite_model_quant)

## Inference

In [ ]:
image_batch, label_batch = val_dataset.as_numpy_iterator().next()
img = image_batch[0,:,:,:]
print (img.flatten())

# Write a C source file of the image for debugging purposes (to check if the model output is correct)
with open('src/img.h', 'w') as f:
    f.write('#ifndef IMG_TEST_DATA\n')
    f.write('#define IMG_TEST_DATA\n')
    f.write('const float img_data[] = {')
    for i in range(0, len(img.flatten())):
        f.write(str(img.flatten()[i]))
        if i < len(img.flatten()) - 1:
            f.write(',')    
    f.write('};\n')
    f.write('#endif\n')



In [ ]:
interpreter = tf.lite.Interpreter(model_path=str(tflite_model_quant_file))
interpreter.allocate_tensors()
input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]
interpreter.set_tensor(input_index, np.expand_dims(img, axis=0))
interpreter.invoke()
output = interpreter.get_tensor(output_index)
print(output.flatten())
if output.flatten()[0] >= 0.5:
    print("Person")
else:
    print("No person")

In [ ]:
plt.imshow(img.astype("uint8"))
print(img.astype("uint8").shape)
print(img.astype("uint8").flatten()[:3])
print(img.astype("float32").flatten()[:3])

In [ ]:
from tqdm import tqdm

def evaluate_model(interpreter):
    input_index = interpreter.get_input_details()[0]["index"]
    output_index = interpreter.get_output_details()[0]["index"]

    prediction_digits = []
    test_labels = []

    for test_image, test_label in tqdm(val_dataset):
        for i in range(len(test_label)):
            interpreter.set_tensor(input_index, np.expand_dims(test_image[i], axis=0))
            interpreter.invoke()
            prediction_digits.append(interpreter.get_tensor(output_index))
            test_labels.append(test_label[i])
    prediction_digits = np.array(prediction_digits).flatten().reshape(-1, 1)
    print(prediction_digits[:10])
    test_labels = np.array(test_labels)
    print(test_labels[:10])
    prediction_digits = np.where(prediction_digits < 0.5, 0, 1)
    print(prediction_digits[:10])

    accuracy = (prediction_digits == test_labels).mean()
    return accuracy

interpreter = tf.lite.Interpreter(model_path=str(tflite_model_file))
interpreter.allocate_tensors()
test_accuracy = evaluate_model(interpreter)

print('Float model accuracy = ', test_accuracy)

interpreter = tf.lite.Interpreter(model_path=str(tflite_model_quant_file))
interpreter.allocate_tensors()
test_accuracy = evaluate_model(interpreter)

print('Quantized model accuracy = ', test_accuracy)


## Generate model CC file

In [66]:
model_file = 'model-int8'
!xxd -i model/{model_file}.tflite | sed "s/unsigned/const unsigned/g" | sed "s/model_model_int8_tflite/gmodel/g"> src/model.cc
# !xxd -n gmodel -i model/{model_file}.tflite | sed "s/unsigned/const unsigned/g" > src/model.cc